In [1]:
from bs4 import BeautifulSoup
import requests
# import requests_async as requests
from fake_useragent import UserAgent
# import asyncio
def getHtml(method, url, params= None, data= None, encoding= 'utf-8'):
    try:
    # headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"}
    # req = requests.request(req_type, url, data)
        headers = {'user-agent':str(UserAgent().random)}

        if method == 'get':
            req = requests.get(url, headers= headers, params= params)
        elif method == 'post':
            req = requests.post(url, headers= headers, data= data)
            
        # req=requests.get(url, headers= headers, params= params)
        if req.status_code == 200:
            print('requests -> 200')
            req.encoding = encoding
            return BeautifulSoup(req.text, 'html.parser')
        print('requests -> ', req.status_code)
        return None
    except:
        return None

In [2]:
def textClean(splitChar='', text='', loop=1, reRight=True, notReverse = True): #
    myIndex = 1 if notReverse else 0
    myLoop = 0
    myRange = text if notReverse else reversed(text)
    for i in myRange:
        if i == splitChar: 
            myLoop+=1
            if myLoop==loop: 
                if reRight:
                    return text[myIndex:] if notReverse else text[-myIndex:] # 由index至右
                return text[:myIndex] if notReverse else text[:-myIndex] # 由左至index
        myIndex+=1

In [3]:
def initIndexSearch(storeFrom, reqKey):
    check = False
    urls = [
        'https://www.books.com.tw/products/%s?sloc=main',
        "https://www.sanmin.com.tw/Product/index/%s"
    ]
    url = urls[storeFrom] % reqKey
    soup = getHtml(method='get', url=url)
    if bool(soup): check = True
    
    # books
    if check:
        infoData = {'title':None, 'isbn':None, 'page':None, 'pub':None, 'author':None, 'date':None, 'price':None}
        if storeFrom == 0: booksIndexSearch(soup=soup , infoData=infoData, intrData=[])
        elif sotoreFrom == 1: 
            sanminIndexSearch(soup)
    else:
        return  0

In [4]:
def books(soup, infoData, intrData):
    # --- Price ---
    infoData['price'] = soup.find('strong',class_='price01').text
    # --- Image URL---
    imgURL = []
    for i in soup.findAll('ul', class_='li_box'):
        for j in i.findAll('li'):
            tmp = j.img['src']
            imgURL.append("%s%s" % (tmp[8:11], tmp.split('/img/')[1].split('&v=')[0]))
    # --- Information ---
    tag = {'書名':'title','ISBN':'isbn','頁數':'page','出版社':'pub','作者':'author', '出版日期':'date'}
    findInfo = soup.find('meta', {'name':'description'})['content']
    for i in findInfo.split('，'):  # book info處理
        tmp = i.split('：')
        if len(tmp)>2: tmp = [tmp[0], textClean(splitChar='：', text=i, loop=1)] # 標題, 內容(防止內容出現':'有問題)
        key = tag[tmp[0]] if (tmp[0] in tag) else None # 確標籤有在需求(tag)裡
        if key: infoData[key] = tmp[1]  # 資料內容
    # --- Intr ---
    soup = soup.find('div', class_="grid_19").findAll('div', class_= 'mod_b type02_m057 clearfix')
    tag = {'內容簡介':'intr', "作者介紹":'author', '目錄':'contents'}
    for i in soup:
        title = i.find('h3').string
        tabTitle = tag[title] if title in tag else None

        findContent = i.find('div', class_='content')
        content = [i.string for i in findContent]
        if key: intrData.append( 
            {
                "title":tabTitle,
                "content":"\n".join(content)
            }
        )
    return {
        'imgURL': imgURL,
        'info': infoData,
        'intr': intrData,
    }
     

In [3]:
itemData = indexSearch("0010858475") 

requests -> 200


In [5]:
url = 'https://www.books.com.tw/products/0010858475?sloc=main'
soup = getHtml(method ='get', url=url)

requests -> 200


In [6]:
test = soup.findAll('ul', class_='li_box')

In [7]:
imgURL = []
for i in test:
#     imgURL.extend(j.img['src'] for j in i.findAll('li'))
    for j in i.findAll('li'):
        tmp = j.img['src']
        imgURL.append("%s%s" % (tmp[8:11], tmp.split('/img/')[1].split('&v=')[0]))
itemData['imgURL']=imgURL

In [19]:
import json
myjson = json.dumps(itemData)

In [20]:
with open("test.json", 'w', encoding='utf-8') as f:
    f.write(myjson)

In [2]:
urls = [
        'https://www.books.com.tw/products/%s?sloc=main',
        "https://www.sanmin.com.tw/Product/index/%s"
    ]
url = urls[0] % 'qqq'
print(url)

https://www.books.com.tw/products/qqq?sloc=main
